# 4. Market Features Engineering
Create features from QQQ (technical indicators), XLK (sector), and VIX (volatility)

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from utils.feature_functions import (
    calculate_returns, calculate_rolling_volatility, calculate_rsi,
    calculate_ma_ratios, calculate_rolling_correlation
)
from utils.hopsworks_helpers import get_feature_store, create_feature_group
import yaml

# Load config
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

## Load Raw Data from Hopsworks

In [ ]:
fs = get_feature_store()

# Load raw feature groups
qqq_fg = fs.get_feature_group('qqq_raw', version=1)
xlk_fg = fs.get_feature_group('xlk_raw', version=1)
vix_fg = fs.get_feature_group('vix_raw', version=1)

qqq_df = qqq_fg.read()
xlk_df = xlk_fg.read()
vix_df = vix_fg.read()

print(f"QQQ data: {qqq_df.shape}")
print(f"XLK data: {xlk_df.shape}")
print(f"VIX data: {vix_df.shape}")

## QQQ Technical Features

In [ ]:
# Calculate returns
qqq_features = calculate_returns(
    qqq_df, 
    price_col='qqq_close',
    periods=config['features']['technical']['return_lags']
)

# Calculate rolling volatility
qqq_features = calculate_rolling_volatility(
    qqq_features,
    price_col='qqq_close',
    windows=config['features']['technical']['volatility_windows']
)

# Calculate RSI
qqq_features = calculate_rsi(
    qqq_features,
    price_col='qqq_close',
    period=config['features']['technical']['rsi_period']
)

# Calculate MA ratios
qqq_features = calculate_ma_ratios(
    qqq_features,
    price_col='qqq_close',
    periods=config['features']['technical']['ma_periods']
)

print(f"QQQ features shape: {qqq_features.shape}")
qqq_features.head()

## XLK Sector Features

In [ ]:
# Calculate XLK returns
xlk_features = calculate_returns(
    xlk_df,
    price_col='xlk_close',
    periods=[1]
)

# Calculate rolling correlation between QQQ and XLK
correlation_df = calculate_rolling_correlation(
    qqq_df,
    xlk_df,
    col1='qqq_close',
    col2='xlk_close',
    windows=config['features']['sector']['correlation_windows']
)

# Merge XLK features with correlation
xlk_features = xlk_features.merge(correlation_df, on='date', how='left')

print(f"XLK features shape: {xlk_features.shape}")
xlk_features.head()

## VIX Volatility Features

In [ ]:
# Calculate VIX daily change
vix_features = vix_df.copy()
vix_features['vix_change'] = vix_features['vix_close'].diff()
vix_features['vix_pct_change'] = vix_features['vix_close'].pct_change()

# Rolling stats on VIX
vix_features['vix_ma_20'] = vix_features['vix_close'].rolling(20).mean()
vix_features['vix_std_20'] = vix_features['vix_close'].rolling(20).std()

print(f"VIX features shape: {vix_features.shape}")
vix_features.head()

## Upload Market Features to Hopsworks

In [ ]:
# Select relevant columns for feature groups
qqq_cols = ['date'] + [col for col in qqq_features.columns if col.startswith(('return_', 'volatility_', 'rsi_', 'ma_ratio_'))]
qqq_features_final = qqq_features[qqq_cols]

xlk_cols = ['date'] + [col for col in xlk_features.columns if col.startswith(('return_', 'corr_'))]
xlk_features_final = xlk_features[xlk_cols]

vix_cols = ['date', 'vix_close', 'vix_change', 'vix_pct_change', 'vix_ma_20', 'vix_std_20']
vix_features_final = vix_features[vix_cols]

In [ ]:
# Create feature groups
qqq_fg = create_feature_group(
    fs,
    name='qqq_technical_features',
    df=qqq_features_final,
    primary_key=['date'],
    description='QQQ technical indicators: returns, volatility, RSI, MA ratios'
)

xlk_fg = create_feature_group(
    fs,
    name='xlk_sector_features',
    df=xlk_features_final,
    primary_key=['date'],
    description='XLK sector features: returns and correlation with QQQ'
)

vix_fg = create_feature_group(
    fs,
    name='vix_volatility_features',
    df=vix_features_final,
    primary_key=['date'],
    description='VIX volatility features: close, change, rolling stats'
)

print("Market features uploaded to Hopsworks!")